In [1]:
import import_hack
import numpy as np
import core.steps as steps
from core import data_store as ds
from core import transformer as tr
from core.common.ds_utils import print_ds_items_info
from os.path import join as pjoin
%load_ext autoreload
%autoreload 2

In [2]:
ds_base_dir=r'C:\data\computation\brodatz\global_descriptors'

# Setup global descriptors datastores
descriptors where computed in global_descriptor__compute_from_image 

In [3]:
histograms_ds = ds.SQLiteTableDataStore(pjoin(ds_base_dir, 'histograms'))
lbphistograms_ds = ds.SQLiteTableDataStore(pjoin(ds_base_dir, 'lbphistograms'))
glcms_ds = ds.SQLiteTableDataStore(pjoin(ds_base_dir, 'glcms'))

# Quantize global descriptors

In [ ]:

quantizer = PQQuantizer(n_clusters=256, n_quantizers=2)

In [30]:
histogram_centroids_ds=SQLiteTableDataStore('ds_data\imgid_histogramclusters256', 'clusterid_cluster')
quantizer = PQQuantizer(n_clusters=256, n_quantizers=2)
steps.quantize_step(histogram_ds, quantizer, histogramclusters_ds)
print_ds_items_info(histogramclusters_ds)

count of items in ds:  2
shape of item[0]:  (256, 128)
shape of item[1]:  (256, 128)


In [24]:
glcmclusters_ds=SQLiteTableDataStore('ds_data\imgid_glcmclusters', 'clusterid_cluster')
quantizer = PQQuantizer(n_clusters=256, n_quantizers=2)
# steps.quantize_step(glcm_ds, quantizer, glcmclusters_ds)
print_ds_items_info(glcmclusters_ds)

count of items in ds:  2
shape of item[0]:  (256, 32768)
shape of item[1]:  (256, 32768)


In [6]:
lbphistogramclusters_ds=SQLiteTableDataStore('ds_data\imgid_lbphistogramclusters50', 'clusterid_cluster')
quantizer = PQQuantizer(n_clusters=50, n_quantizers=2)
steps.quantize_step(lbphistogram_ds, quantizer, lbphistogramclusters_ds)
print_ds_items_info(lbphistogramclusters_ds)

count of items in ds:  2
shape of item[0]:  (50, 256)
shape of item[1]:  (50, 256)


### Build inverted multi-index

In [31]:
histogramclusters_ndarray_ds=StreamNdarrayAdapterDataStore(histogramclusters_ds, detect_final_shape_by_first_elem=True)
histgoramcluster_centers=histogramclusters_ndarray_ds.get_items_sorted_by_ids()
print("histgoramcluster_centers: ", histgoramcluster_centers.shape)
histogram_ndarray_ds = StreamNdarrayAdapterDataStore(histogram_ds, detect_final_shape_by_first_elem=True)
histogram_X = histogram_ndarray_ds.get_items_sorted_by_ids()
histogram_X_ids=histogram_ndarray_ds.get_ids_sorted()
print("X: ", histogram_X.shape)
print("X_ids: ", histogram_X_ids.shape)
histogram_imi_searcher=InvertedMultiIndexSearcher(histogram_X, histogram_X_ids, histgoramcluster_centers)

histgoramcluster_centers:  (2, 256, 128)
X:  (999, 256, 1)
X_ids:  (999,)


In [28]:
glcmclusters_ndarray_ds=StreamNdarrayAdapterDataStore(glcmclusters_ds, detect_final_shape_by_first_elem=True)
glcmcluster_centers=glcmclusters_ndarray_ds.get_items_sorted_by_ids()
glcm_ndarray_ds = StreamNdarrayAdapterDataStore(glcm_ds, detect_final_shape_by_first_elem=True)
glcm_X = glcm_ndarray_ds.get_items_sorted_by_ids()
glcm_X_ids=glcm_ndarray_ds.get_ids_sorted()
glcm_imi_searcher=InvertedMultiIndexSearcher(glcm_X, glcm_X_ids, glcmcluster_centers)

In [8]:
lbphistogramclusters_ndarray_ds=StreamNdarrayAdapterDataStore(lbphistogramclusters_ds, detect_final_shape_by_first_elem=True)
lbphistgoramcluster_centers=lbphistogramclusters_ndarray_ds.get_items_sorted_by_ids()
lbphistogram_ndarray_ds = StreamNdarrayAdapterDataStore(lbphistogram_ds, detect_final_shape_by_first_elem=True)
lbphistogram_X = lbphistogram_ndarray_ds.get_items_sorted_by_ids()
lbphistogram_X_ids=lbphistogram_ndarray_ds.get_ids_sorted()
lbphistogram_imi_searcher=InvertedMultiIndexSearcher(lbphistogram_X, lbphistogram_X_ids, lbphistgoramcluster_centers)

### Search approximate neighbors for all bovws(productsiftbincount_ds)

In [32]:
# %%timeit -n3 -r3
histogram_imi_neighborsids_ds=NumpyDataStore()
steps.search_step(histogram_ds, histogram_imi_searcher, n_nearest, histogram_imi_neighborsids_ds)

In [33]:
print_ds_items_info(histogram_imi_neighborsids_ds, print_shape=False, first_items_to_print=5)

count of items in ds:  999
item[0]:  [  3   1   8   5   9   2   6   4 332  98  97  95  94  92  91 331 998 997
 994 477 755 754 648 647 644]
item[1]:  [  9   2   6   3   1   8   5  98  97  95  94  92  91 332 477 331  99  96
  93 538 648 647 644 642 641]
item[2]:  [  3   1   9   2   6   8   5  98  97  95  94  92  91 332   4 477 331 998
 997 994 755 754 648 647 644]
item[3]:  [  4   7   8   5   3   1 998 997 994 995 991 332 755 754 999 992 261 648
 647 644 642 641 333 258 331]
item[4]:  [  8   5   3   1   4   9   2   6   7 332  98  97  95  94  92  91 998 997
 994 331 648 647 644 642 641]


In [21]:
glcm_approximateneighborsids_ds=NumpyDataStore()
steps.search_step(glcm_ds, glcm_imi_searcher, n_nearest, glcm_approximateneighborsids_ds)
print_ds_items_info(glcm_approximateneighborsids_ds, print_shape=False, first_items_to_print=5)

NameError: name 'glcm_ds' is not defined

In [154]:
# %%timeit -n3 -r3
lbphistogram_imi_neighborsids_ds=NumpyDataStore()
steps.search_step(lbphistogram_ds, lbphistogram_imi_searcher, n_nearest, lbphistogram_imi_neighborsids_ds)
# print_ds_items_info(lbphistogram_imi_neighborsids_ds, print_shape=False, first_items_to_print=5)

In [155]:
histogram_imi_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(histogram_imi_neighborsids_ds, perfomance_evaluator, histogram_imi_retrieval_perfomance_ds)

In [156]:
lbphistogram_imi_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(lbphistogram_imi_neighborsids_ds, perfomance_evaluator, lbphistogram_imi_retrieval_perfomance_ds)

In [157]:
histogram_imi_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(histogram_imi_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds)

In [158]:
lbphistogram_imi_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(lbphistogram_imi_retrieval_perfomance_ds, lbphistogram_imi_average_retrieval_perfomance_ds)

In [159]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels, 
             1, 'precision', loc='lower left')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 1.          0.97797798  0.95395395  0.92992993  0.90610611  0.87771104
  0.85113685  0.81969469  0.77922367  0.71741742  0.66511967  0.62137137
  0.58235158  0.54733305  0.51531532  0.48736236  0.46311017  0.44088533
  0.41999895  0.4007007   0.38324038  0.36727637  0.35291814  0.33921421
  0.32656657]
[ 1.          0.93643644  0.88688689  0.8455956   0.80660661  0.77360694
  0.73902474  0.70720721  0.67534201  0.62612613  0.58312858  0.54562896
  0.51274351  0.48305448  0.45699032  0.43374625  0.41253018  0.39328217
  0.37532269  0.35955956  0.34491634  0.33092183  0.31844888  0.30697364
  0.29641642]
[ 0.97097097  0.94294294  0.91324658  0.88288288  0.85065065  0.81715048
  0.78149578  0.74537037  0.71249027  0.65515516  0.60578761  0.56456456
  0.52837453  0.49714     0.469336    0.44413163  0.42271683  0.40279168
  0.38480586  0.36806807  0.35244769  0.3

In [12]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels, 
             2, 
             'recall',
             loc='lower left')

NameError: name 'plot_results' is not defined

In [104]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels,
             3, 'mean average precision',
             loc='lower left')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 0.1         0.1955956   0.28575242  0.37031198  0.44955122  0.52087254
  0.58682421  0.64308046  0.68525597  0.6968876   0.70644261  0.71507625
  0.72165975  0.72646456  0.72972115  0.73303696  0.73662879  0.73943715
  0.74119681  0.7424931   0.74371337  0.74483267  0.74605564  0.74685226
  0.74755297]
[ 0.1         0.18728729  0.26459793  0.3344678   0.39624958  0.45293961
  0.50137375  0.54492981  0.580343    0.59308574  0.60261345  0.61007091
  0.61593832  0.62047857  0.62476286  0.6287043   0.63166019  0.63432953
  0.6363368   0.63834381  0.64005981  0.64122916  0.64261751  0.64388544
  0.64499455]
[ 0.0970971   0.18778779  0.27140474  0.34845679  0.41812646  0.47962129
  0.53281734  0.57781234  0.61768554  0.62692478  0.63366789  0.63974897
  0.64455377  0.64920128  0.65300508  0.65584542  0.65909573  0.66153706
  0.66387098  0.66573784  0.66715354  0.6